In [ ]:
import cvxpy as cp    ## To install cvxpy, https://www.cvxpy.org/install/index.html
import numpy as np

from scipy.linalg import sqrtm

# <center> Toy Model </center>

## 1. State Preparation

$$
q_0 = 1/3, |\psi_0\rangle = |0\rangle\\
q_1 = 1/3, |\psi_1\rangle = |1\rangle\\
q_2 = 1/3, |\psi_2\rangle = |+\rangle
$$

In [ ]:
q_list = [1/3] * 3
rho_0 = np.array([[1,0],[0,0]])
rho_1 = np.array([[0,0],[0,1]])
rho_2 = np.array([[1/2,1/2],[1/2,1/2]])
rho_list = [rho_0, rho_1,rho_2]

## 2. Semi-Definite Programming for MED

### Primal Problem

$$
\max \sum_{i=0}^{l-1}q_i\text{Tr}[E_i\rho_i]\\
\text{Subject to } E_i \ge 0, \forall i=1,\cdots, l \\
\sum^{l-1}_{i=0}E_i=I
$$

In [ ]:
# Create l 2x2 matrix variables
E = [cp.Variable((2,2), hermitian=True) for x in range(3)]

# Create constraints
#constraints = [E[0] + E[1] + E[2] == np.eye(2)]

sum_all_E = 0
for i in range(3):
    sum_all_E += E[i]
"""sum_all_E = E[0]
for i in range(1, 3):
    sum_all_E += E[i]"""
constraints = [sum_all_E == np.eye(2)]

constraints += [
    E[i] >> 0 for i in range(3)
]

# Form objective.
q_rho = [q_list[i] * rho_list[i] for i in range(3)]
obj = 1 - cp.real(cp.trace(E[0] @ q_rho[0]) + cp.trace(E[1] @ q_rho[1]) + cp.trace(E[2] @ q_rho[2]))

prob = cp.Problem(cp.Minimize(obj), constraints)
prob.solve()

print("status:", prob.status)
print("optimal value", prob.value)
print("A solution E is")
for i in range(3):
    print("E_"+str(i)+" =", E[i].value)


WARN: A->p (column pointers) not strictly increasing, column 9 empty
WARN: A->p (column pointers) not strictly increasing, column 12 empty
WARN: A->p (column pointers) not strictly increasing, column 13 empty
WARN: A->p (column pointers) not strictly increasing, column 16 empty
WARN: A->p (column pointers) not strictly increasing, column 17 empty
WARN: A->p (column pointers) not strictly increasing, column 20 empty
status: optimal
optimal value 0.3333333577148129
A solution E is
E_0 = [[9.99999967e-01+0.j 4.20319352e-09+0.j]
 [4.20319352e-09+0.j 3.12216186e-08+0.j]]
E_1 = [[3.12216184e-08+0.j 4.20319352e-09+0.j]
 [4.20319352e-09+0.j 9.99999967e-01+0.j]]
E_2 = [[ 2.01313412e-09+0.j -8.45401309e-09+0.j]
 [-8.45401309e-09+0.j  2.01313472e-09+0.j]]


### Dual Problem

$$
\min_{K} \text{Tr}[K]\\
\text{Subject to } K - q_i\rho_i \ge 0, \forall i=1,\cdots, l
$$

In [ ]:
K = cp.Variable((2,2), hermitian=True)
constraints = [K - q_rho[i] >> 0 for i in range(3)]
prob = cp.Problem(cp.Minimize(cp.real(cp.trace(K))), constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)
print("A solution K is")
print(K.value)

WARN: A->p (column pointers) not strictly increasing, column 3 empty
WARN: A->p (column pointers) not strictly increasing, column 6 empty
status: optimal
optimal value 0.6666643491044055
A solution K is
[[3.33332175e-01+0.j 3.72752713e-06+0.j]
 [3.72752713e-06+0.j 3.33332175e-01+0.j]]


## 3. Pretty Good Measurement

$$
\Pi_i = q_i \rho^{-1/2} \rho_i \rho^{-1/2} \text{, where } \rho = \sum_i q_i\rho_i
$$

In this example, $\rho = \frac{1}{3}\left(|0\rangle\langle 0| + |1\rangle\langle 1| + |+\rangle\langle +| \right)$. By the spectral decomposition,
$$
\rho = \begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ 
\frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}}
\end{pmatrix}
\begin{pmatrix}
\frac{2}{3} & 0 \\ 
0 & \frac{1}{3}
\end{pmatrix}
\begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ 
\frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}}
\end{pmatrix}
$$

In [ ]:
rho_cvsum = np.array([[1/2,1/6],[1/6,1/2]])
rho_cvsum

array([[0.5       , 0.16666667],
       [0.16666667, 0.5       ]])

$$
\rho^{1/2} = \begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ 
\frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}}
\end{pmatrix}
\begin{pmatrix}
\sqrt{\frac{2}{3}} & 0 \\ 
0 & \sqrt{\frac{1}{3}}
\end{pmatrix}
\begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ 
\frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}}
\end{pmatrix}
=
\frac{1}{2\sqrt{3}}
\begin{pmatrix}
\sqrt{2} + 1 & \sqrt{2} - 1 \\ 
\sqrt{2} - 1 & \sqrt{2} + 1
\end{pmatrix}
$$

In [ ]:
sqrt_rho = sqrtm(rho_cvsum)
sqrt_rho

array([[0.69692343, 0.11957316],
       [0.11957316, 0.69692343]])

$$
\rho^{-1/2} = \frac{1}{\det \rho^{1/2}}
\frac{1}{2\sqrt{3}}
\begin{pmatrix}
\sqrt{2} + 1 & 1 - \sqrt{2}  \\ 
1 - \sqrt{2} & \sqrt{2} + 1
\end{pmatrix}
$$

In [ ]:
inv_sqrt_rho = np.linalg.inv(sqrt_rho)
inv_sqrt_rho

array([[ 1.47839784, -0.25365297],
       [-0.25365297,  1.47839784]])

$$
\Pi_i = q_i \rho^{-1/2} \rho_i \rho^{-1/2}
$$

In [ ]:
def pgm(q_list, rho_list):
    l = len(q_list)
    rho_cvsum = np.sum([q_list[i] * rho_list[i] for i in range(l)], axis=0)

    sqrt_rho = sqrtm(rho_cvsum)
    inv_sqrt_rho = np.linalg.inv(sqrt_rho)

    pgm_list = [q_list[i] * np.dot(np.dot(inv_sqrt_rho, rho_list[i]), inv_sqrt_rho) for i in range(l)]
    return pgm_list

In [ ]:
pgm_list = pgm(q_list, rho_list)
pgm_list

[array([[ 0.72855339, -0.125     ],
        [-0.125     ,  0.02144661]]),
 array([[ 0.02144661, -0.125     ],
        [-0.125     ,  0.72855339]]),
 array([[0.25, 0.25],
        [0.25, 0.25]])]

$$
p_{i|i} = \text{Tr}[\Pi_i\rho_i]
$$

In [ ]:
prob_list = [np.trace(np.dot(pgm_list[i], rho_list[i])) for i in range(3)]
prob_list

[0.7285533905932741, 0.7285533905932741, 0.5000000000000002]

$$
p_{error} = 1 - \sum_i q_ip_{i|i}
$$

In [ ]:
error_prob = 1 - np.sum([q_list[i] * prob_list[i] for i in range(3)])
error_prob

0.34763107293781725

# <center> Semi-Definite Program for POVM classification </center>

## 1. Codes

In [46]:
class POVM_Clf_SDP():
    def __init__(self, dim_povm=2, num_povms=2, problem_type='Primal'):
        self.problem = problem_type
        self.dim = dim_povm
        self.num = num_povms


    def Primal(self):
        # Create num dim x dim matrix variables
        E = [cp.Variable((self.dim, self.dim), hermitian=True) for x in range(self.num)]

        # Create constraints
        ## Equality Constraints
        sum_all_E = 0
        for i in range(self.num):
            sum_all_E += E[i]
        constraints = [sum_all_E == np.eye(self.dim)]
        ## Inequality Constraints
        constraints += [
            E[i] >> 0 for i in range(self.num)
        ]

        # Form an objective function.
        obj = 0
        for i in range(self.num):
            obj += cp.real(cp.trace(E[i] @ self.q_rho[i]))

        prob = cp.Problem(cp.Maximize(obj), constraints)
        prob.solve()

        print("Opt is Done. \nStatus:", prob.status)
        E_opt = [E[i].value for i in range(self.num)]
        med_value = 1 - prob.value

        return med_value, E_opt

    def Dual(self):

        K = cp.Variable((self.dim,self.dim), hermitian=True)
        constraints = [K - self.q_rho[i] >> 0 for i in range(self.num)]
        prob = cp.Problem(cp.Minimize(cp.real(cp.trace(K))), constraints)
        prob.solve()

        print("Opt is Done. \nStatus:", prob.status)
        K_opt = K.value
        med_value = 1 - prob.value

        return med_value, K_opt

    def __call__(self, init_states, a_priori_probs):
        
        if self.num != len(init_states):
            raise ValueError("The number of POVMS != The number of quantum states")

        elif self.dim != len(init_states[0]):
            raise ValueError("The dimension of POVM != The dimension of quantum state")

        

        self.init_rhos = [np.outer(init_states[i], init_states[i]) for i in range(self.num)]
        self.q_rho = [a_priori_probs[i] * self.init_rhos[i] for i in range(self.num)]

        return self.Primal() if self.problem == 'Primal' else self.Dual()
        


## 2. Examples

In [1]:
import numpy as np
from povm_clf import POVM_Clf_SDP

### i) 
$\{|0\rangle, |1\rangle, |+\rangle \}$

In [12]:
q_list = [1/3] * 3
states = [[1,0], [0,1], [1/np.sqrt(2), 1/np.sqrt(2)]]

Primal_povm_clf = POVM_Clf_SDP(2, 3, 'Primal')
Dual_povm_clf = POVM_Clf_SDP(2, 3, 'Dual')

In [13]:
Primal_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 9 empty
WARN: A->p (column pointers) not strictly increasing, column 12 empty
WARN: A->p (column pointers) not strictly increasing, column 13 empty
WARN: A->p (column pointers) not strictly increasing, column 16 empty
WARN: A->p (column pointers) not strictly increasing, column 17 empty
WARN: A->p (column pointers) not strictly increasing, column 20 empty
Opt is Done. 
Status: optimal


(0.33333335771481254,
 [array([[9.99999967e-01+0.j, 4.20319375e-09+0.j],
         [4.20319375e-09+0.j, 3.12216178e-08+0.j]]),
  array([[3.12216167e-08+0.j, 4.20319375e-09+0.j],
         [4.20319375e-09+0.j, 9.99999967e-01+0.j]]),
  array([[ 2.01313528e-09+0.j, -8.45401373e-09+0.j],
         [-8.45401373e-09+0.j,  2.01313493e-09+0.j]])])

In [14]:
Dual_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 3 empty
WARN: A->p (column pointers) not strictly increasing, column 6 empty
Opt is Done. 
Status: optimal


(0.33333565089559447,
 array([[3.33332175e-01+0.j, 3.72752713e-06+0.j],
        [3.72752713e-06+0.j, 3.33332175e-01+0.j]]))

### ii) 
$\{|0\rangle,|+\rangle\}$

In [21]:
q_list = [1/2] * 2
states = [[1,0], [1/np.sqrt(2),1/np.sqrt(2)]]

Primal_povm_clf = POVM_Clf_SDP(2, 2, 'Primal')
Dual_povm_clf = POVM_Clf_SDP(2, 2, 'Dual')

In [22]:
Primal_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 6 empty
WARN: A->p (column pointers) not strictly increasing, column 9 empty
WARN: A->p (column pointers) not strictly increasing, column 10 empty
WARN: A->p (column pointers) not strictly increasing, column 13 empty
Opt is Done. 
Status: optimal


(0.14644540518881366,
 [array([[ 0.85355459+0.j, -0.35355459+0.j],
         [-0.35355459+0.j,  0.14644541+0.j]]),
  array([[0.14644541+0.j, 0.35355459+0.j],
         [0.35355459+0.j, 0.8535546 +0.j]])])

In [23]:
Dual_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 3 empty
WARN: A->p (column pointers) not strictly increasing, column 6 empty
Opt is Done. 
Status: optimal


(0.14644675787549621,
 array([[0.55177654+0.j, 0.12500008+0.j],
        [0.12500008+0.j, 0.3017767 +0.j]]))

### ii)' 
$\{|0\rangle,|+i\rangle\}$

In [2]:
q_list = [1/2] * 2
states = [[1,0], [1/np.sqrt(2),1j/np.sqrt(2)]]

Primal_povm_clf = POVM_Clf_SDP(2, 2, 'Primal')
Dual_povm_clf = POVM_Clf_SDP(2, 2, 'Dual')

In [3]:
Primal_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 6 empty
WARN: A->p (column pointers) not strictly increasing, column 9 empty
WARN: A->p (column pointers) not strictly increasing, column 10 empty
WARN: A->p (column pointers) not strictly increasing, column 13 empty
Opt is Done. 
Status: optimal


(0.14644531701461339,
 [array([[8.53554682e-01+0.j        , 7.78596098e-17+0.35355468j],
         [7.78596098e-17-0.35355468j, 1.46445322e-01+0.j        ]]),
  array([[ 1.46445324e-01+0.j        , -7.83894222e-17-0.35355468j],
         [-7.83894222e-17+0.35355468j,  8.53554684e-01+0.j        ]])])

In [4]:
Dual_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 3 empty
WARN: A->p (column pointers) not strictly increasing, column 6 empty
Opt is Done. 
Status: optimal


(0.14644677966789488,
 array([[ 5.51776307e-01+0.j       , -8.77001689e-18-0.1250003j],
        [-8.77001689e-18+0.1250003j,  3.01776913e-01+0.j       ]]))

### iii) 
$\{(|0\rangle+|1\rangle)/\sqrt{2},\}$

In [2]:
q_list = [1/3] * 3
states = [[1/np.sqrt(2), np.exp(1j*2*np.pi/3)/np.sqrt(2)], [1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), np.exp(-1j*2*np.pi/3)/np.sqrt(2)]]

Primal_povm_clf = POVM_Clf_SDP(2, 3, 'Primal')
Dual_povm_clf = POVM_Clf_SDP(2, 3, 'Dual')

In [3]:
Primal_povm_clf(states, q_list)

Opt is Done. 
Status: optimal


(0.33333235460073685,
 [array([[ 0.33333333+0.j        , -0.16666716-0.28867598j],
         [-0.16666716+0.28867598j,  0.33333333+0.j        ]]),
  array([[0.33333333+0.00000000e+00j, 0.33333431-1.01176007e-16j],
         [0.33333431+1.01176007e-16j, 0.33333333+0.00000000e+00j]]),
  array([[ 0.33333333+0.j        , -0.16666716+0.28867598j],
         [-0.16666716-0.28867598j,  0.33333333+0.j        ]])])

In [4]:
Dual_povm_clf(states, q_list)

Opt is Done. 
Status: optimal


(0.3333333135926595,
 array([[ 3.33333343e-01+0.00000000e+00j, -1.48335316e-16-8.17623199e-16j],
        [-1.48335316e-16+8.17623199e-16j,  3.33333343e-01+0.00000000e+00j]]))

### iv)
$\{|00\rangle, |0+\rangle, |+0\rangle, |++\rangle \}$

In [9]:
states = [[1,0,0,0], [1/np.sqrt(2),0,1/np.sqrt(2),0], [1/np.sqrt(2),1/np.sqrt(2),0,0],[1/2,1/2,1/2,1/2]]
q_list = [1/len(states)] * len(states)

Primal_povm_clf = POVM_Clf_SDP(4, 4, 'Primal')
Dual_povm_clf = POVM_Clf_SDP(4, 4, 'Dual')

In [10]:
Primal_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 40 empty
WARN: A->p (column pointers) not strictly increasing, column 45 empty
WARN: A->p (column pointers) not strictly increasing, column 50 empty
WARN: A->p (column pointers) not strictly increasing, column 55 empty
WARN: A->p (column pointers) not strictly increasing, column 56 empty
WARN: A->p (column pointers) not strictly increasing, column 61 empty
WARN: A->p (column pointers) not strictly increasing, column 66 empty
WARN: A->p (column pointers) not strictly increasing, column 71 empty
WARN: A->p (column pointers) not strictly increasing, column 72 empty
WARN: A->p (column pointers) not strictly increasing, column 77 empty
WARN: A->p (column pointers) not strictly increasing, column 82 empty
WARN: A->p (column pointers) not strictly increasing, column 87 empty
WARN: A->p (column pointers) not strictly increasing, column 88 empty
WARN: A->p (column pointers) not strictly increasing, column 93 empty
WARN: A->p (column p

(0.2714502647128738,
 [array([[ 0.72854973+0.j, -0.30177432+0.j, -0.30177432+0.j,
           0.12499887+0.j],
         [-0.30177432+0.j,  0.12500089+0.j,  0.12499895+0.j,
          -0.05177631+0.j],
         [-0.30177432+0.j,  0.12499895+0.j,  0.12500089+0.j,
          -0.05177631+0.j],
         [ 0.12499887+0.j, -0.05177631+0.j, -0.05177631+0.j,
           0.02144847+0.j]]),
  array([[ 0.12500099+0.j, -0.05177641+0.j,  0.30177442+0.j,
          -0.12499897+0.j],
         [-0.05177641+0.j,  0.02144837+0.j, -0.12499905+0.j,
           0.05177621+0.j],
         [ 0.30177442+0.j, -0.12499905+0.j,  0.72854964+0.j,
          -0.30177423+0.j],
         [-0.12499897+0.j,  0.05177621+0.j, -0.30177423+0.j,
           0.12500099+0.j]]),
  array([[ 0.12500099+0.j,  0.30177442+0.j, -0.05177641+0.j,
          -0.12499897+0.j],
         [ 0.30177442+0.j,  0.72854964+0.j, -0.12499905+0.j,
          -0.30177423+0.j],
         [-0.05177641+0.j, -0.12499905+0.j,  0.02144837+0.j,
           0.05177621+0.

In [11]:
Dual_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 10 empty
WARN: A->p (column pointers) not strictly increasing, column 15 empty
WARN: A->p (column pointers) not strictly increasing, column 20 empty
WARN: A->p (column pointers) not strictly increasing, column 25 empty
Opt is Done. 
Status: optimal


(0.27143939486234003,
 array([[0.30445934+0.j, 0.06897205+0.j, 0.06897205+0.j, 0.0156249 +0.j],
        [0.06897205+0.j, 0.16651517+0.j, 0.01562491+0.j, 0.03772215+0.j],
        [0.06897205+0.j, 0.01562491+0.j, 0.16651517+0.j, 0.03772215+0.j],
        [0.0156249 +0.j, 0.03772215+0.j, 0.03772215+0.j, 0.09107093+0.j]]))

### v)
$\{|00\rangle+|11\rangle, |00\rangle-|11\rangle, |01\rangle+|10\rangle, |01\rangle-|10\rangle \}$

In [12]:
states = [[1/np.sqrt(2),0,0,1/np.sqrt(2)], [1/np.sqrt(2),0,0,-1/np.sqrt(2)], [0,1/np.sqrt(2),1/np.sqrt(2),0], [0,1/np.sqrt(2),-1/np.sqrt(2),0]]
q_list = [1/len(states)] * len(states)

Primal_povm_clf = POVM_Clf_SDP(4, 4, 'Primal')
Dual_povm_clf = POVM_Clf_SDP(4, 4, 'Dual')

In [13]:
Primal_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 40 empty
WARN: A->p (column pointers) not strictly increasing, column 45 empty
WARN: A->p (column pointers) not strictly increasing, column 50 empty
WARN: A->p (column pointers) not strictly increasing, column 55 empty
WARN: A->p (column pointers) not strictly increasing, column 56 empty
WARN: A->p (column pointers) not strictly increasing, column 61 empty
WARN: A->p (column pointers) not strictly increasing, column 66 empty
WARN: A->p (column pointers) not strictly increasing, column 71 empty
WARN: A->p (column pointers) not strictly increasing, column 72 empty
WARN: A->p (column pointers) not strictly increasing, column 77 empty
WARN: A->p (column pointers) not strictly increasing, column 82 empty
WARN: A->p (column pointers) not strictly increasing, column 87 empty
WARN: A->p (column pointers) not strictly increasing, column 88 empty
WARN: A->p (column pointers) not strictly increasing, column 93 empty
WARN: A->p (column p

(-4.66527661635574e-06,
 [array([[ 5.00001556e-01+0.j, -1.11076643e-16+0.j,  7.28368844e-18+0.j,
           5.00003109e-01+0.j],
         [-1.11076643e-16+0.j, -1.55323986e-06+0.j, -1.27319474e-16+0.j,
          -7.62671749e-17+0.j],
         [ 7.28368844e-18+0.j, -1.27319474e-16+0.j, -1.55323986e-06+0.j,
          -3.79206088e-18+0.j],
         [ 5.00003109e-01+0.j, -7.62671749e-17+0.j, -3.79206088e-18+0.j,
           5.00001556e-01+0.j]]),
  array([[ 5.00001556e-01+0.j,  9.16093206e-17+0.j, -8.37054227e-18+0.j,
          -5.00003109e-01+0.j],
         [ 9.16093206e-17+0.j, -1.55323986e-06+0.j, -1.27319474e-16+0.j,
          -2.20440503e-17+0.j],
         [-8.37054227e-18+0.j, -1.27319474e-16+0.j, -1.55323987e-06+0.j,
          -3.27044501e-18+0.j],
         [-5.00003109e-01+0.j, -2.20440503e-17+0.j, -3.27044501e-18+0.j,
           5.00001556e-01+0.j]]),
  array([[-1.55323986e-06+0.j,  1.05403593e-17+0.j, -7.12016063e-18+0.j,
           1.34876379e-16+0.j],
         [ 1.05403593e-17+0

In [14]:
Dual_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 10 empty
WARN: A->p (column pointers) not strictly increasing, column 15 empty
WARN: A->p (column pointers) not strictly increasing, column 20 empty
WARN: A->p (column pointers) not strictly increasing, column 25 empty
Opt is Done. 
Status: optimal


(4.478725195045108e-06,
 array([[ 2.49998880e-01+0.j,  2.48115733e-16+0.j,  1.06557146e-16+0.j,
         -5.92755312e-16+0.j],
        [ 2.48115733e-16+0.j,  2.49998880e-01+0.j, -6.65103565e-16+0.j,
         -8.09537819e-16+0.j],
        [ 1.06557146e-16+0.j, -6.65103565e-16+0.j,  2.49998880e-01+0.j,
          2.76576339e-16+0.j],
        [-5.92755312e-16+0.j, -8.09537819e-16+0.j,  2.76576339e-16+0.j,
          2.49998880e-01+0.j]]))

### vi)
$\{|00\rangle+|11\rangle, |01\rangle+|10\rangle, |00\rangle, |++\rangle \}$

In [13]:
states = [[1/np.sqrt(2),0,0,1/np.sqrt(2)], [0,1/np.sqrt(2),1/np.sqrt(2),0], [1,0,0,0], [1/2,1/2,1/2,1/2]]
q_list = [1/len(states)] * len(states)

Primal_povm_clf = POVM_Clf_SDP(4, 4, 'Primal')
Dual_povm_clf = POVM_Clf_SDP(4, 4, 'Dual')

In [14]:
med_val, E_opt = Primal_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 40 empty
WARN: A->p (column pointers) not strictly increasing, column 45 empty
WARN: A->p (column pointers) not strictly increasing, column 50 empty
WARN: A->p (column pointers) not strictly increasing, column 55 empty
WARN: A->p (column pointers) not strictly increasing, column 56 empty
WARN: A->p (column pointers) not strictly increasing, column 61 empty
WARN: A->p (column pointers) not strictly increasing, column 66 empty
WARN: A->p (column pointers) not strictly increasing, column 71 empty
WARN: A->p (column pointers) not strictly increasing, column 72 empty
WARN: A->p (column pointers) not strictly increasing, column 77 empty
WARN: A->p (column pointers) not strictly increasing, column 82 empty
WARN: A->p (column pointers) not strictly increasing, column 87 empty
WARN: A->p (column pointers) not strictly increasing, column 88 empty
WARN: A->p (column pointers) not strictly increasing, column 93 empty
WARN: A->p (column p

In [15]:
Dual_povm_clf(states, q_list)

WARN: A->p (column pointers) not strictly increasing, column 10 empty
WARN: A->p (column pointers) not strictly increasing, column 15 empty
WARN: A->p (column pointers) not strictly increasing, column 20 empty
WARN: A->p (column pointers) not strictly increasing, column 25 empty
Opt is Done. 
Status: optimal


(0.3232354240828901,
 array([[2.75887733e-01+0.j, 9.43428758e-06+0.j, 9.43428758e-06+0.j,
         6.24974130e-02+0.j],
        [9.43428758e-06+0.j, 1.24997153e-01+0.j, 1.24997039e-01+0.j,
         1.14830793e-05+0.j],
        [9.43428758e-06+0.j, 1.24997039e-01+0.j, 1.24997153e-01+0.j,
         1.14830793e-05+0.j],
        [6.24974130e-02+0.j, 1.14830793e-05+0.j, 1.14830793e-05+0.j,
         1.50882538e-01+0.j]]))